# JON'S HiML Competition XGB_Log Model (v 3.0)
## Make Date: 04/13/18
Can XGBoost improve our prediction accuracy?  Use model 2.2, except use XGB instead of random forest.

In [3]:
#Some initialization procedures:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split

import xgboost as xgb

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

# load in data files
# FILE_DIR = '../input/hawaiiml-data'
# for f in os.listdir(FILE_DIR):
#     print('{0:<30}{1:0.2f}MB'.format(f, 1e-6*os.path.getsize(f'{FILE_DIR}/{f}')))


FILE_DIR = '../Sprint09alt_Machine_Learning_Hawaii_Kaggle_Competition'
df_train = pd.read_csv(f'{FILE_DIR}/train.csv', encoding='ISO-8859-1') #write training data to dataframe
df_test = pd.read_csv(f'{FILE_DIR}/test.csv', encoding='ISO-8859-1') # Read the test data

#define the error function:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_true) - np.log1p(y_pred))**2))

# Prediction Target
We want to predict the quantity data field.    
By convention, we define this target as 'y'.  
We know quantity is highly skewed. See if a log transform will help.  

In [4]:
y = df_train.quantity
logy = np.log1p(df_train.quantity)
df_train['log1p_quantity'] = np.log1p(df_train.quantity)

# Train Model
setup XGB using these site:   
  https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/  
  https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/  
For now, just try out numerics - drop all nonnumerics


In [5]:
df_train.drop(['date','time','country','description'], axis=1, inplace=True)
df_test.drop(['date','time','country','description'], axis=1, inplace=True)

In [6]:
def TrainModel(ls_AllPredictors, df_X, logy, myModel):
    #use passed in myModel: train, make prediction, and then evaluate predition.
    train_X, val_X, train_y, val_y = train_test_split(df_X[ls_AllPredictors], logy, random_state = 0) #split training data into a test and train part
    myModel.fit(train_X, train_y)  
    # make predictions for test data
    pred_y = myModel.predict(val_X)
    # predictions = [round(value) for value in pred_y]
    err = rmsle(np.expm1(val_y), np.expm1(pred_y)) #include transform of val_Y values from log to "normal" Y value  
    return err, myModel

In [18]:
def leftORRight():
    ls_leftORright = [-1,1]
    return ls_leftORright[random.randint(0,1)]
#     return -1 #for now, force left

def HopToVal(HopFactor, CurVal, r_vals):
    #hop to another value within range R_vals that is at most HopFactor% away from current value
    #returned value is float so you'll need to round and cast (Say to int) if needed
    xrMin = r_vals[0]
    xrMax = r_vals[len(r_vals)-1]
    LR = leftORRight()
    if LR < 0:
        HopDistMax = xrMin-CurVal
    else:
        HopDistMax = xrMax-CurVal
    x1 = CurVal + HopDistMax * HopFactor * random.uniform(0,HopFactor)
    if x1 < xrMin: x1=xrMin
    return x1
    
def prAccept(prevErr, curErr, T):
    try:
        prob = math.exp((prevErr - curErr)/T)
    except OverflowError:
        prob = 1 #calculated number is infinite. make prob = 1
    return prob

def AssignModelParams(i_n_estimators, i_max_depth, SeedVal):
    #set model values using passed in parameters. use predetermined values for everythin else
    myModel = xgb.XGBRegressor(
        learning_rate =0.1,
        n_estimators=i_n_estimators,
        max_depth=i_max_depth,
        early_stopping_rounds=5,
        nthread=8,
        seed=SeedVal)  
    return myModel
       
def FindBestModel(ls_AllPredictors, df_X, logy, 
                 MaxWalks, 
                 r_n_estimators, r_max_depth,
                 SeedVal
                 ):
    #tune till we find the best set of model parameters
    #FOR NOW DO A SIMPLISTIC RANDOM OPTIMIZATION SEARCH THAT DOES RANDOM HOPS. IF HOP LOCATION IMPROVES ERROR, THEN USE THAT HOP AS JUMP POINT FOR NEXT HOP
    #AS WE PROCEED TO TOWARD MAXWALK, HOP DIST APPROACHES 0
    bestErr = 100000000
    i_n_estimators = r_n_estimators[0]
    i_max_depth = r_max_depth[0] 
    T = 1
    for WalkNo in range (0, MaxWalks):
        myModel = AssignModelParams(i_n_estimators, i_max_depth, SeedVal)
        curErr, myModel = TrainModel(ls_AllPredictors, df_X, logy, myModel)
        print ('Cur trial:', WalkNo, '  Trained with params: myModel.n_estimators', myModel.n_estimators, 'myModel.max_depth', myModel.max_depth, ' ERR:',curErr)
        if curErr < bestErr:
            bestModel = myModel
            bestErr = curErr
            print ('!!!!NEW BEST: err:', bestErr)
            #use last estimator as hop off point for next parameter
            i_n_estimators = int(round(HopToVal(T,i_n_estimators,r_n_estimators),0))
            i_max_depth = int(round(HopToVal(T,i_max_depth, r_max_depth),0))
        else:
            #use best estimator as hop off point for next parameter
            i_n_estimators = int(round(HopToVal(T,bestModel.n_estimators,r_n_estimators),0))
            i_max_depth = int(round(HopToVal(T,bestModel.max_depth, r_max_depth),0))            
        T = 1 - WalkNo/MaxWalks
    return bestErr, bestModel

In [17]:
ls_AllPredictors = ['id','invoice_id', 'stock_id', 'customer_id', 'unit_price']
bestErr, bestModel = FindBestModel(ls_AllPredictors, df_train, logy, 14, range(1,5000), range(6,30),0)

TypeError: __init__() got an unexpected keyword argument 'early_stopping_rounds'

# Submit Model's Predictions
## First, output model's predictions for test data set:

In [11]:
# Use the model to make predictions
predicted_vals = bestModel.predict(df_test)
predicted_vals = np.expm1(bestModel.predict(df_test)) #transform from log to "normal" Y value
# We will look at the predicted prices to ensure we have something sensible.
print(predicted_vals)

[4.260911  7.2110157 9.191854  ... 2.867107  1.5157086 2.127908 ]


## Next, submit predicted values

In [ ]:
my_submission = pd.DataFrame({'Id': df_test.id, 'quantity': predicted_vals})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)